In [1]:
from sklearn.datasets import load_digits
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

In [2]:
digits = load_digits()

data_tensor = torch.tensor(digits.data, dtype=torch.float32)
target_tensor = F.one_hot(torch.tensor(digits.target), num_classes=10).float()

# 80/20 split
X_train, X_test, y_train, y_test = train_test_split(
    data_tensor, target_tensor, test_size=0.2, random_state=42, shuffle=True
)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

In [5]:
# Simple FFN approach

class FFN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(8*8, 32),
            nn.ReLU(),
            nn.Linear(32, 10),
            # nn.ReLU(),
            # nn.Linear(512, 10),
        )

    def forward(self, x):
        # x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
        return F.softmax(logits, dim=0)

model = FFN()

epochs = 100

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for i in range(epochs):
    for data_batch, label_batch in train_loader:
        output = model(data_batch)
        loss = loss_fn(output, label_batch)
    #     total_loss += loss.item() * data_batch.size(0)
    #     total_samples += data_batch.size(0)
                    
    # for j in range(0, len(data_tensor), batch_size):
    #     pred = model(data_tensor[j:j+batch_size])
        
    #     target = target_tensor[j:j+batch_size]

        # loss = loss_fn(pred, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if i % 10 == 0:
        print(loss.item())

1.6766685247421265
0.08708133548498154
0.03224492818117142
0.016007812693715096
0.009106047451496124
0.005902233999222517
0.003920753486454487
0.0027513038367033005
0.0020087866578251123
0.0015113046392798424


In [6]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, one_hot_labels in test_loader:
        data, one_hot_labels = data, one_hot_labels

        outputs = model(data)  # logits
        predicted = torch.argmax(outputs, dim=1)  # class indices
        true_labels = torch.argmax(one_hot_labels, dim=1)  # class indices from one-hot

        correct += (predicted == true_labels).sum().item()
        total += true_labels.size(0)

accuracy = correct / total
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 96.94%
